In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('missing').getOrCreate()

In [4]:
tranning=spark.read.csv(r'C:\Users\Asus\Desktop\MyCode\code\code\PYthon_only\ETL\data\test1.csv',header=True,inferSchema=True)


In [5]:
tranning.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [6]:
tranning.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [7]:
tranning.columns

['Name', 'age', 'Experience', 'Salary']

In [ ]:
[Age,Experience]

In [9]:
from pyspark.ml.feature import VectorAssembler


In [10]:
featureassembler=VectorAssembler(inputCols=['age','Experience'],outputCol='independent feature')


In [11]:
output=featureassembler.transform(tranning)

In [13]:
output.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|independent feature|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



In [14]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'independent feature']

In [15]:
finalized_data=output.select('independent feature','Salary')

In [16]:
finalized_data.show()

+-------------------+------+
|independent feature|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



In [17]:
from pyspark.ml.regression import LinearRegression
#train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regrssor=LinearRegression(featuresCol='independent feature',labelCol='Salary')
regrssor=regrssor.fit(train_data)

In [18]:
regrssor.coefficients

DenseVector([750.0, 1500.0])

In [19]:
#intercepts
regrssor.intercept

-2249.9999999995152

In [20]:
pred_results=regrssor.evaluate(test_data)

In [22]:
#prediction
pred_results.predictions.show()

+-------------------+------+------------------+
|independent feature|Salary|        prediction|
+-------------------+------+------------------+
|         [24.0,3.0]| 20000|20249.999999999938|
|         [29.0,4.0]| 20000| 25499.99999999982|
|         [30.0,8.0]| 25000|32249.999999999756|
|        [31.0,10.0]| 30000| 35999.99999999971|
+-------------------+------+------------------+



In [23]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(4749.999999999806, 29718749.99999775)